# Creazione dataset e test di MusicGen

In [ ]:
!pip install transformers omegaconf num2words julius

from transformers import pipeline

In [ ]:
import soundfile as sf
from google.colab import drive


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Prova di MusicGen
generator = pipeline("text-to-audio", model="facebook/musicgen-small") #Richiamo MusicGen
prompt = "drift phonk" #qui scrivo il prompt da dare al modello
output = generator(prompt, forward_params={"max_length": 256}) #passo il prompt al modello

# Extract audio + sample rate
audio = output["audio"].squeeze() # levo dimensione extra
sr = output["sampling_rate"] #prende il sampling rate da ciò che ha generato il modello

filename = f"/content/drive/MyDrive/audio-restoration/generated_samples_by_musicgen/{prompt.replace(' ', '_')}.wav"
sf.write(filename, audio, sr) #salva il file generato nel percorso descritto da filename
print(f"✅ Saved to Drive: {filename}")

# salva pure in locale su Colab
sf.write("generated_music_local.wav", audio, sr)

Device set to use cpu


✅ Saved to Drive: /content/drive/MyDrive/audio-restoration/generated_samples_by_musicgen/drift_phonk.wav


# Creazione Dataset

In [ ]:
import librosa
import numpy as np
import soundfile as sf
from tqdm import tqdm

In [ ]:
# @title Download Dataset

!pip install -q spotdl

from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

dataset_dir = "/content/drive/MyDrive/audio-restoration/data/raw"
os.makedirs(dataset_dir, exist_ok=True)

# Scarica Phonk playlist
print("\n📥 Download Phonk Mix...")
!spotdl download \
  "https://open.spotify.com/playlist/5wUqmRSrdK2ax6EHJkSzD8?si=89c075a3f96e4073" \
  --output {dataset_dir} \
  --format wav

# Verifica
from pathlib import Path
files = list(Path(dataset_dir).glob("*.wav"))
print(f"\n✅ Total downloaded: {len(files)} files")
print(f"Total size: {sum(f.stat().st_size for f in files) / 1e9:.2f} GB")


In [ ]:
# Aumenta timeout
import time
time.sleep(2)  # Piccolo delay tra requests (lagga di meno colab)

In [ ]:
# Scarica Top 100 Global
print("📥 Downloading Top 100 Global")
!spotdl download \
  "https://open.spotify.com/playlist/0sDahzOkMWOmLXfTMf2N4N?si=61cb837dd5574d4e" \
  --output {dataset_dir} \
  --format wav